In [1]:
from astropy.table import Table
import numpy as np


#import the cartons
#read in data file
#SAVE COLUMNS TO NUMPY ARRAYS
cartdata = Table.read("BHM_cartons_COSMOS_v0.5.fits", format='fits')

#the arrays, the only ones we really care about are catid and priority
catalogid = np.array(cartdata["catalogid"], dtype=np.int64)
ra = np.array(cartdata["ra"])
dec = np.array(cartdata["dec"])
target_pk = np.array(cartdata["target_pk"], dtype=np.int64)
priority = np.array(cartdata["priority"])
value = np.array(cartdata["value"])
carton = np.array(cartdata["carton"],dtype=str )
instrument = np.array(cartdata["instrument"], dtype='<U10')
carton_to_target_pk = np.array(cartdata['carton_to_target_pk'])
g = np.array(cartdata['g'])
r = np.array(cartdata['r'])
i = np.array(cartdata['i'])
bp = np.array(cartdata['bp'])
gaia_g = np.array(cartdata['gaia_g'])
rp = np.array(cartdata['rp'])
h = np.array(cartdata['h'])
len(carton)

3419

In [2]:
catid = 27021598150214584
carton[np.where(catalogid == catid)]

array(['bhm_rm_core'], dtype='<U17')

In [4]:
#import the design file
file= "COSMOS_design1.fits"
data2 = Table.read(file, hdu = 2, format='fits')
data1 = Table.read(file, hdu = 1, format='fits')

#make array of what non-knownspec targets were used

assigned_other = ((data2['assigned'] == 1) & (data1['category'] == 'science') & (data1['carton'] != 'bhm_rm_known_spec'))
assigned_SCIENCEfibers = ((data2['assigned'] == 1) & (data1['category'] == 'science'))
assigned_CALIBfibers = ((data2['assigned'] == 1) & ((data1['category'] == 'sky_boss') | (data1['category'] == 'standard_boss')))


assigned_known = ((data2['assigned'] == 1) & (data1['carton'] == 'bhm_rm_known_spec'))
assigned_core = ((data2['assigned'] == 1) & (data1['carton'] == 'bhm_rm_core'))
assigned_var = ((data2['assigned'] == 1) & (data1['carton'] == 'bhm_rm_var'))
assigned_ancil = ((data2['assigned'] == 1) & (data1['carton'] == 'bhm_rm_ancillary'))
threshold_mask = ((data1['carton'][assigned_SCIENCEfibers] == 'bhm_rm_known_spec') | (data1["priority"][assigned_SCIENCEfibers] <= 950.))

print("------- COSMOS Design 1 --------")
print("calib: ", len(data1[assigned_CALIBfibers]))
print("science: ", len(data1[assigned_SCIENCEfibers]))
print("total assignments: {}/500".format(len(data1[assigned_CALIBfibers]) + len(data1[assigned_SCIENCEfibers])))


assert len(data1[assigned_other]) == (len(data1[assigned_SCIENCEfibers]) - len(data1[assigned_known])), "not equal!"
print("")
print("science breakdown")
# print("known spec: ", len(data1[assigned_known]))
# print("other (var+core+ancil): ",len(data1[assigned_other]))
# print("var: ", len(data1[assigned_var]))
# print("core: ", len(data1[assigned_core]))
# print("ancil: ", len(data1[assigned_ancil]))
print("known QSOs: ", len(data1['priority'][assigned_SCIENCEfibers][threshold_mask]))
print("candidate QSOs: ", len(data1['priority'][assigned_SCIENCEfibers][~threshold_mask]))

------- COSMOS Design 1 --------
calib:  120
science:  371
total assignments: 491/500

science breakdown
known QSOs:  245
candidate QSOs:  126


In [3]:
#investigate the current priorities of the cartons from the fits file
print("unique core priorities: ", np.unique(priority[np.where(carton == 'bhm_rm_core')]))
print("unique var priorities: ", np.unique(priority[np.where(carton == 'bhm_rm_var')]))
print("unique ancillary priorities: ", np.unique(priority[np.where(carton == 'bhm_rm_ancillary')]))

unique core priorities:  [ 912.  917.  922.  927.  932.  937.  942. 1007. 1012. 1017. 1022. 1027.
 1032. 1037. 1042. 1047.]
unique var priorities:  [ 913.  918.  923.  928.  933.  938. 1008. 1013. 1018. 1023. 1028. 1033.
 1038.]
unique ancillary priorities:  [ 914.  919.  924.  929.  934.  939. 1009. 1014. 1019. 1024. 1029. 1034.
 1039. 1044.]


In [4]:
#check known_spec priority to ensure I understand which way it needs to go
print("unique knownspec priorities: ", np.unique(priority[np.where(carton == 'bhm_rm_known_spec')]))

unique knownspec priorities:  [ 901.  941. 1001. 1041. 1046.]


In [5]:
#investigate the priorities of the targets used in the design file
print("unique targ priorities: ",np.unique(data1['priority'][assigned_other]))

unique targ priorities:  priority
--------
    1012
    1022
    1023
    1027
    1028
    1032
    1033
    1037
    1038
    1042
    1047


In [6]:
#iterate through catIDs and reset priorities of those targets 
#try resetting to like 900, if that doesn't work, then just the lowest value in that carton
#currently resetting every value for that target, not just the carton used in the design

#NEEDS HIGHER PRIORITY, COUNTERINTUITIVE

##FUTURE MEG: CHANGE THIS
print("catid     carton     priority   ")
for i in range(len(data1["catalogid"][assigned_other])):
    print(data1["catalogid"][assigned_other][i], data1["carton"][assigned_other][i], data1["priority"][assigned_other][i])
    #IMPLEMENT TIERED REASSIGNMENT BASED ON CARTON, CORE > VAR > ANCILLARY, remember counterintutiveness
    priority[np.where((catalogid == data1["catalogid"][assigned_other][i])|(data1["priority"][assigned_other][i] > 950.))] += 1000.
    print(priority[np.where((catalogid == data1["catalogid"][assigned_other][i]))])#&(carton == data1["carton"][assigned_other][i]))])

catid     carton     priority   
27021598150214584 bhm_rm_core 1032
[2032.]
27021598150248096 bhm_rm_core 1032
[2032.]
27021598150222509 bhm_rm_core 1032
[2032. 2034.]
27021598150226161 bhm_rm_core 1037
[2037. 2039. 2038.]
27021598150210733 bhm_rm_core 1027
[2027.]
27021598150222218 bhm_rm_core 1037
[2037. 2039.]
27021598150288383 bhm_rm_core 1032
[2032.]
27021598150266987 bhm_rm_core 1037
[2037. 2039. 2038.]
27021598150247939 bhm_rm_core 1037
[2037.]
27021598150226198 bhm_rm_core 1032
[2032. 2034. 2033.]
27021598150244913 bhm_rm_core 1027
[2027. 2029.]
27021598150266959 bhm_rm_core 1032
[2032.]
27021598150266494 bhm_rm_core 1037
[2037.]
27021598150202099 bhm_rm_core 1012
[2012.]
27021598150219168 bhm_rm_core 1012
[2012.]
27021600814741956 bhm_rm_core 1022
[2022.]
27021598150223578 bhm_rm_core 1022
[2022. 2023.]
27021598150213499 bhm_rm_core 1022
[2022.]
27021598150226157 bhm_rm_core 1022
[2022.]
27021598150220233 bhm_rm_core 1022
[2022.]
27021598150223965 bhm_rm_core 1022
[2022.]
2702

In [10]:
# #generate new design with these priorities, copy from script?
# #or just save a new carton fits file?
# #let's try saving a new carton fits?
# from astropy.table import Table, Column
# tab = Table()
# tab.add_column(Column(name="catalogid", data=catalogid, dtype='int64'))
# tab.add_column(Column(name="ra", data=ra, dtype='float32'))
# tab.add_column(Column(name="dec", data=dec, dtype='float32'))
# tab.add_column(Column(name="target_pk", data=target_pk, dtype='int64'))
# tab.add_column(Column(name="priority", data=priority, dtype='float32'))
# tab.add_column(Column(name="value", data=value, dtype='float32'))
# tab.add_column(Column(name="carton", data=carton))
# tab.add_column(Column(name="instrument", data=instrument, dtype='<U10'))
# tab.add_column(Column(name="carton_to_target_pk", data=carton_to_target_pk, dtype='int64'))
# tab.add_column(Column(name="g", data=g, dtype='float32'))
# tab.add_column(Column(name="r", data=r, dtype='float32'))
# tab.add_column(Column(name="i", data=i, dtype='float32'))
# tab.add_column(Column(name="bp", data=bp, dtype='float32'))
# tab.add_column(Column(name="gaia_g", data=gaia_g, dtype='float32'))
# tab.add_column(Column(name="rp", data=rp, dtype='float32'))
# tab.add_column(Column(name="h", data=h, dtype='float32'))



# tab.write("BHM_COSMOS_v0.5_DEPRIORITY1.fits", format='fits', overwrite=True)


In [5]:
#check targets, compare
#import the deprioritized design
dep_file= "COSMOS_design2.fits"
data_deprior2 = Table.read(dep_file, hdu = 2, format='fits')
data_deprior1 = Table.read(dep_file, hdu = 1, format='fits')

Dassigned_other = ((data_deprior2['assigned'] == 1) & (data_deprior1['category'] == 'science') & (data_deprior1['carton'] != 'bhm_rm_known_spec'))
Dassigned_SCIENCEfibers = ((data_deprior2['assigned'] == 1) & (data_deprior1['category'] == 'science'))
Dassigned_CALIBfibers = ((data_deprior2['assigned'] == 1) & ((data_deprior1['category'] == 'sky_boss') | (data_deprior1['category'] == 'standard_boss')))

Dassigned_known = ((data_deprior2['assigned'] == 1) & (data_deprior1['carton'] == 'bhm_rm_known_spec'))
Dassigned_core = ((data_deprior2['assigned'] == 1) & (data_deprior1['carton'] == 'bhm_rm_core'))
Dassigned_var = ((data_deprior2['assigned'] == 1) & (data_deprior1['carton'] == 'bhm_rm_var'))
Dassigned_ancil = ((data_deprior2['assigned'] == 1) & (data_deprior1['carton'] == 'bhm_rm_ancillary'))
Dthreshold_mask = ((data_deprior1['carton'][Dassigned_SCIENCEfibers] == 'bhm_rm_known_spec') | (data_deprior1["priority"][Dassigned_SCIENCEfibers] <= 950.))

print("------- COSMOS Design 2 --------")
print("calib: ", len(data_deprior1[Dassigned_CALIBfibers]))
print("science: ", len(data_deprior1[Dassigned_SCIENCEfibers]))
print("total assignments: {}/500".format( len(data_deprior1[Dassigned_CALIBfibers]) + len(data_deprior1[Dassigned_SCIENCEfibers])))
assert len(data_deprior1[Dassigned_other]) == (len(data_deprior1[Dassigned_SCIENCEfibers]) - len(data_deprior1[Dassigned_known])), "not equal!"
print("")
print("science breakdown")
# print("known spec: ", len(data_deprior1[Dassigned_known]))
# print("other (var+core+ancil): ",len(data_deprior1[Dassigned_other]))
# print("var: ", len(data_deprior1[Dassigned_var]))
# print("core: ", len(data_deprior1[Dassigned_core]))
# print("ancil: ", len(data_deprior1[Dassigned_ancil]))
print("known QSOs: ", len(data_deprior1['priority'][Dassigned_SCIENCEfibers][Dthreshold_mask]))
print("candidate QSOs: ", len(data_deprior1['priority'][Dassigned_SCIENCEfibers][~Dthreshold_mask]))

------- COSMOS Design 2 --------
calib:  120
science:  373
total assignments: 493/500

science breakdown
known QSOs:  257
candidate QSOs:  116


In [12]:
#other target (not known spec) overlap between the two designs
targ_overlap = np.array(list(set(data1["catalogid"][assigned_other]) & set(data_deprior1['catalogid'][Dassigned_other])))
len(targ_overlap)

14

In [13]:
targ_overlap = np.array(list(set(data1["catalogid"][assigned_known]) & set(data_deprior1['catalogid'][Dassigned_known])))
len(targ_overlap)
#len(data1["catalogid"][assigned_known])

245

In [14]:
#Round two of deprioritization
print("catid     carton     priority   ")
for i in range(len(data_deprior1["catalogid"][Dassigned_other])):
    print(data_deprior1["catalogid"][Dassigned_other][i], data_deprior1["carton"][Dassigned_other][i], data_deprior1["priority"][Dassigned_other][i])
    priority[np.where((catalogid == data_deprior1["catalogid"][Dassigned_other][i]))] += 1000.
    print(priority[np.where((catalogid == data_deprior1["catalogid"][Dassigned_other][i]))])

catid     carton     priority   
27021598150243913 bhm_rm_core 1037
[2037.]
27021598150247725 bhm_rm_core 1042
[2042.]
27021598150222509 bhm_rm_core 2032
[3032. 3034.]
27021598150226200 bhm_rm_core 1037
[2037. 2039. 2038.]
27021598150266951 bhm_rm_core 1042
[2042.]
27021598150217966 bhm_rm_core 1037
[2037.]
27021598150218615 bhm_rm_core 1042
[2042.]
27021598150225616 bhm_rm_core 1042
[2042.]
27021598150224205 bhm_rm_core 1037
[2037.]
27021598150210778 bhm_rm_core 1037
[2037.]
27021598150266654 bhm_rm_core 1042
[2042.]
27021598150218334 bhm_rm_core 1042
[2042.]
27021598150266315 bhm_rm_core 1032
[2032.]
27021600049986617 bhm_rm_core 1027
[2027.]
27021600069291127 bhm_rm_core 1027
[2027.]
27021600129429477 bhm_rm_core 1032
[2032.]
27021600129427199 bhm_rm_core 1032
[2032.]
27021600106195355 bhm_rm_core 1032
[2032.]
27021598150209053 bhm_rm_core 1037
[2037.]
27021598150219255 bhm_rm_core 1037
[2037.]
27021600053400334 bhm_rm_core 1037
[2037.]
27021600812783302 bhm_rm_core 1037
[2037.]
270

In [15]:
# from astropy.table import Table, Column
# tab = Table()
# tab.add_column(Column(name="catalogid", data=catalogid, dtype='int64'))
# tab.add_column(Column(name="ra", data=ra, dtype='float32'))
# tab.add_column(Column(name="dec", data=dec, dtype='float32'))
# tab.add_column(Column(name="target_pk", data=target_pk, dtype='int64'))
# tab.add_column(Column(name="priority", data=priority, dtype='float32'))
# tab.add_column(Column(name="value", data=value, dtype='float32'))
# tab.add_column(Column(name="carton", data=carton))
# tab.add_column(Column(name="instrument", data=instrument, dtype='<U10'))
# tab.add_column(Column(name="carton_to_target_pk", data=carton_to_target_pk, dtype='int64'))
# tab.add_column(Column(name="g", data=g, dtype='float32'))
# tab.add_column(Column(name="r", data=r, dtype='float32'))
# tab.add_column(Column(name="i", data=i, dtype='float32'))
# tab.add_column(Column(name="bp", data=bp, dtype='float32'))
# tab.add_column(Column(name="gaia_g", data=gaia_g, dtype='float32'))
# tab.add_column(Column(name="rp", data=rp, dtype='float32'))
# tab.add_column(Column(name="h", data=h, dtype='float32'))



# tab.write("BHM_COSMOS_v0.5_DEPRIORITY2.fits", format='fits', overwrite=True)

In [6]:
#check targets, compare
#import the deprioritized design
dep_file= "COSMOS_design3.fits"
data_deprior22 = Table.read(dep_file, hdu = 2, format='fits')
data_deprior12 = Table.read(dep_file, hdu = 1, format='fits')

Dassigned_other2 = ((data_deprior22['assigned'] == 1) & (data_deprior12['category'] == 'science') & (data_deprior12['carton'] != 'bhm_rm_known_spec'))
Dassigned_SCIENCEfibers2 = ((data_deprior22['assigned'] == 1) & (data_deprior12['category'] == 'science'))
Dassigned_CALIBfibers2 = ((data_deprior22['assigned'] == 1) & ((data_deprior12['category'] == 'sky_boss') | (data_deprior12['category'] == 'standard_boss')))
Dassigned_known2 = ((data_deprior22['assigned'] == 1) & (data_deprior12['carton'] == 'bhm_rm_known_spec'))
Dassigned_core2 = ((data_deprior22['assigned'] == 1) & (data_deprior12['carton'] == 'bhm_rm_core'))
Dassigned_var2 = ((data_deprior22['assigned'] == 1) & (data_deprior12['carton'] == 'bhm_rm_var'))
Dassigned_ancil2 = ((data_deprior22['assigned'] == 1) & (data_deprior12['carton'] == 'bhm_rm_ancillary'))
Dthreshold_mask2 = ((data_deprior12['carton'][Dassigned_SCIENCEfibers2] == 'bhm_rm_known_spec') | (data_deprior12["priority"][Dassigned_SCIENCEfibers2] <= 950.))


print("------- COSMOS Design 3 --------")
print("calib: ", len(data_deprior12[Dassigned_CALIBfibers2]))
print("science: ", len(data_deprior12[Dassigned_SCIENCEfibers2]))
print("total assignments: {}/500".format( len(data_deprior12[Dassigned_CALIBfibers2]) + len(data_deprior12[Dassigned_SCIENCEfibers2])))


assert len(data_deprior12[Dassigned_other2]) == (len(data_deprior12[Dassigned_SCIENCEfibers2]) - len(data_deprior12[Dassigned_known2])), "not equal!"
print("")
print("science breakdown")
# print("known spec: ", len(data_deprior12[Dassigned_known2]))
# print("other (var+core+ancil): ",len(data_deprior12[Dassigned_other2]))
# print("var: ", len(data_deprior12[Dassigned_var2]))
# print("core: ", len(data_deprior12[Dassigned_core2]))
# print("ancil: ", len(data_deprior12[Dassigned_ancil2]))
print("known QSOs: ", len(data_deprior12['priority'][Dassigned_SCIENCEfibers2][Dthreshold_mask2]))
print("candidate QSOs: ", len(data_deprior12['priority'][Dassigned_SCIENCEfibers2][~Dthreshold_mask2]))

------- COSMOS Design 3 --------
calib:  120
science:  373
total assignments: 493/500

science breakdown
known QSOs:  269
candidate QSOs:  104


In [17]:
kstarg_overlap = np.array(list(set(data_deprior12['catalogid'][Dassigned_known2]) & set(data1['catalogid'][assigned_known]) & set(data_deprior1['catalogid'][Dassigned_known])))
len(kstarg_overlap)

242

In [10]:
#other target (not known spec) overlap between the two designs
total_targ_overlap = np.array(list(set(data_deprior12['catalogid'][Dassigned_other2]) & set(data_deprior1['catalogid'][Dassigned_other]) & set(data1['catalogid'][assigned_other])))
targ_overlap_12 = np.array(list(set(data_deprior1['catalogid'][Dassigned_other]) & set(data1['catalogid'][assigned_other])))
targ_overlap_23 = np.array(list(set(data_deprior12['catalogid'][Dassigned_other2]) & set(data_deprior1['catalogid'][Dassigned_other])))
targ_overlap_13 = np.array(list(set(data_deprior12['catalogid'][Dassigned_other2]) &  set(data1['catalogid'][assigned_other])))
kstarg_overlap = np.array(list(set(data_deprior12['catalogid'][Dassigned_known2]) & set(data1['catalogid'][assigned_known]) & set(data_deprior1['catalogid'][Dassigned_known])))
kstarg_overlap_12 = np.array(list(set(data_deprior1['catalogid'][Dassigned_known]) & set(data1['catalogid'][assigned_known])))
kstarg_overlap_23 = np.array(list(set(data_deprior12['catalogid'][Dassigned_known2]) & set(data_deprior1['catalogid'][Dassigned_known])))
kstarg_overlap_13 = np.array(list(set(data_deprior12['catalogid'][Dassigned_known2]) &  set(data1['catalogid'][assigned_known])))



print("COSMOS common non-known_spec targets across:")
print("      all three designs: ",len(total_targ_overlap))
print("                1 and 2: ",len(targ_overlap_12))
print("                2 and 3: ", len(targ_overlap_23))
print("                1 and 3: ", len(targ_overlap_13))
print("")
print("COSMOS common known_spec across:")
print("      all three designs: ", len(kstarg_overlap))
print("                1 and 2: ",len(kstarg_overlap_12))
print("                2 and 3: ", len(kstarg_overlap_23))
print("                1 and 3: ", len(kstarg_overlap_13))

COSMOS common non-known_spec targets across:
      all three designs:  12
                1 and 2:  14
                2 and 3:  14
                1 and 3:  25

COSMOS common known_spec across:
      all three designs:  241
                1 and 2:  244
                2 and 3:  254
                1 and 3:  242


In [11]:
##NEW FORMAT

#other target (not known spec) overlap between the two designs
total_targ_overlap = np.array(list(set(data_deprior12['catalogid'][Dassigned_SCIENCEfibers2][~Dthreshold_mask2]) & set(data_deprior1['catalogid'][Dassigned_SCIENCEfibers][~Dthreshold_mask]) & set(data1['catalogid'][assigned_SCIENCEfibers][~threshold_mask])))
targ_overlap_12 = np.array(list(set(data_deprior1['catalogid'][Dassigned_SCIENCEfibers][~Dthreshold_mask]) & set(data1['catalogid'][assigned_SCIENCEfibers][~threshold_mask])))
targ_overlap_23 = np.array(list(set(data_deprior12['catalogid'][Dassigned_SCIENCEfibers2][~Dthreshold_mask2]) & set(data_deprior1['catalogid'][Dassigned_SCIENCEfibers][~Dthreshold_mask])))
targ_overlap_13 = np.array(list(set(data_deprior12['catalogid'][Dassigned_SCIENCEfibers2][~Dthreshold_mask2]) &  set(data1['catalogid'][assigned_SCIENCEfibers][~threshold_mask])))

kstarg_overlap = np.array(list(set(data_deprior12['catalogid'][Dassigned_SCIENCEfibers2][Dthreshold_mask2]) & set(data_deprior1['catalogid'][Dassigned_SCIENCEfibers][Dthreshold_mask]) & set(data1['catalogid'][assigned_SCIENCEfibers][threshold_mask])))
kstarg_overlap_12 = np.array(list(set(data_deprior1['catalogid'][Dassigned_SCIENCEfibers][Dthreshold_mask]) & set(data1['catalogid'][assigned_SCIENCEfibers][threshold_mask])))
kstarg_overlap_23 = np.array(list(set(data_deprior12['catalogid'][Dassigned_SCIENCEfibers2][Dthreshold_mask2]) & set(data_deprior1['catalogid'][Dassigned_SCIENCEfibers][Dthreshold_mask])))
kstarg_overlap_13 = np.array(list(set(data_deprior12['catalogid'][Dassigned_SCIENCEfibers2][Dthreshold_mask2]) &  set(data1['catalogid'][assigned_SCIENCEfibers][threshold_mask])))



print("COSMOS common candidate QSOs targets across:")
print("      all three designs: ",len(total_targ_overlap))
print("                1 and 2: ",len(targ_overlap_12))
print("                2 and 3: ", len(targ_overlap_23))
print("                1 and 3: ", len(targ_overlap_13))
print("")
print("COSMOS common known QSOs across:")
print("      all three designs: ", len(kstarg_overlap))
print("                1 and 2: ",len(kstarg_overlap_12))
print("                2 and 3: ", len(kstarg_overlap_23))
print("                1 and 3: ", len(kstarg_overlap_13))

COSMOS common candidate QSOs targets across:
      all three designs:  12
                1 and 2:  14
                2 and 3:  14
                1 and 3:  25

COSMOS common known QSOs across:
      all three designs:  241
                1 and 2:  244
                2 and 3:  254
                1 and 3:  242
